In [10]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# --- Define Data ---
demand_data = pd.read_excel('Realistic Data Demand.xlsx')
demand = demand_data['Demand'].tolist()
T = len(demand)  # 自动设置时间段数
generators = ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8']

min_uptime = {'G1': 168, 'G2': 24, 'G3': 24, 'G4': 12, 'G5': 8, 'G6': 8, 'G7': 0, 'G8': 0}
min_downtime = {'G1': 168, 'G2': 12, 'G3': 12, 'G4': 8, 'G5': 4, 'G6': 4, 'G7': 0, 'G8': 0}
initial_on_time = {'G1': 85, 'G2': 0, 'G3': 20, 'G4': 15, 'G5': 6, 'G6': 5, 'G7': 0, 'G8': 0}
initial_off_time = {'G1': 0, 'G2': 10, 'G3': 0, 'G4': 0, 'G5': 0, 'G6': 0, 'G7': 3, 'G8': 12}
min_capacity = {'G1': 240, 'G2': 235, 'G3': 210, 'G4': 32, 'G5': 480, 'G6': 195, 'G7': 0, 'G8': 0}
max_capacity = {'G1': 480, 'G2': 590, 'G3': 520, 'G4': 406, 'G5': 870, 'G6': 350, 'G7': 735, 'G8': 1410}
startup_cost = {'G1': 10380, 'G2': 33590, 'G3': 0, 'G4': 23420, 'G5': 0, 'G6': 0, 'G7': 0, 'G8': 0}
no_load_cost = {'G1': 0, 'G2': 530, 'G3': 490, 'G4': 395, 'G5': 830, 'G6': 255, 'G7': 0, 'G8': 0}
variable_cost = {'G1': 7.7, 'G2': 16.3, 'G3': 17, 'G4': 23.7, 'G5': 40, 'G6': 45, 'G7': 75, 'G8': 77}

lambda_penalty = 100  #

pRE = demand_data['Renewable Scenario 2'].tolist()
alpha = 0.5

# --- Create Gurobi Model ---
model = gp.Model('Unit Commitment Takriti 2000')

# --- Define Decision Variables ---
u = model.addVars(generators, range(1, T+1), vtype=GRB.BINARY, name='u')
p = model.addVars(generators, range(1, T+1), vtype=GRB.CONTINUOUS, name='p')
s = model.addVars(range(1, T+1), vtype=GRB.CONTINUOUS, name='s')
v = model.addVars(generators, range(1, T+1), vtype=GRB.BINARY, name='v')
# --- Set Objective Function ---
model.setObjective(
    gp.quicksum(no_load_cost[g] * u[g, t] + variable_cost[g] * p[g, t] + startup_cost[g] * v[g, t]
                for g in generators for t in range(1, T+1)) +
    lambda_penalty * gp.quicksum(s[t] for t in range(1, T+1)),
    GRB.MINIMIZE
)

# --- Add Constraints ---

# 1. Power Output Constraints
for g in generators:
    for t in range(1, T+1):
        model.addConstr(p[g, t] >= min_capacity[g] * u[g, t], name=f'min_cap_{g}_{t}')
        model.addConstr(p[g, t] <= max_capacity[g] * u[g, t], name=f'max_cap_{g}_{t}')

# 2. Demand Satisfaction
for t in range(1, T+1):
    model.addConstr(
        gp.quicksum(p[g, t] for g in generators) + alpha * pRE[t - 1] == demand[t - 1] + s[t],
        name=f'demand_{t}'
    )
    model.addConstr(s[t] >= 0, name=f's_nonneg_{t}')

# 3. Minimum Uptime Constraints
for g in generators:
    u_0 = 1 if initial_on_time[g] > 0 else 0
    # t=1
    s_max = min(min_uptime[g] - 1, T - 1)
    for ss in range(1, s_max + 1):
        model.addConstr(u[g, 1] - u_0 <= u[g, 1 + ss], name=f'min_up_{g}_1_{1+ss}')
    # t=2 到 T-1
    for t in range(2, T):
        s_max = min(min_uptime[g] - 1, T - t)
        for ss in range(1, s_max + 1):
            model.addConstr(u[g, t] - u[g, t-1] <= u[g, t + ss], name=f'min_up_{g}_{t}_{t+ss}')

# 4. Minimum Downtime Constraints
for g in generators:
    u_0 = 1 if initial_on_time[g] > 0 else 0
    # t=1
    s_max = min(min_downtime[g] - 1, T - 1)
    for ss in range(1, s_max + 1):
        model.addConstr(u_0 - u[g, 1] <= 1 - u[g, 1 + ss], name=f'min_down_{g}_1_{1+ss}')
    # t=2 到 T-1
    for t in range(2, T):
        s_max = min(min_downtime[g] - 1, T - t)
        for ss in range(1, s_max + 1):
            model.addConstr(u[g, t-1] - u[g, t] <= 1 - u[g, t + ss], name=f'min_down_{g}_{t}_{t+ss}')# 5. Startup Definition Constraints
for g in generators:
    # For t = 2, ..., T
    for t in range(2, T+1):
        model.addConstr(v[g, t] >= u[g, t] - u[g, t-1], name=f'start_def_{g}_{t}')
    # For t = 1, use initial status u_{i,0}
    u_prev = 1 if initial_on_time[g] > 0 else 0
    model.addConstr(v[g, 1] >= u[g, 1] - u_prev, name=f'start_def_{g}_1')

# 6. Initial Condition Constraints
for g in generators:
    # If u_{i,0} = 1 and U_1^i < L_i, enforce u_{i,t} = 1
    if initial_on_time[g] > 0 and initial_on_time[g] < min_uptime[g]:
        remaining_uptime = min_uptime[g] - initial_on_time[g]
        for t in range(1, min(remaining_uptime, T) + 1):
            model.addConstr(u[g, t] == 1, name=f'initial_on_{g}_{t}')
    # If u_{i,0} = 0 and U_0^i < l_i, enforce u_{i,t} = 0
    if initial_off_time[g] > 0 and initial_off_time[g] < min_downtime[g]:
        remaining_downtime = min_downtime[g] - initial_off_time[g]
        for t in range(1, min(remaining_downtime, T) + 1):
            model.addConstr(u[g, t] == 0, name=f'initial_off_{g}_{t}')

# --- Solve the Model ---
model.optimize()

# --- Output Results ---
if model.status == GRB.OPTIMAL:
    print(f"Optimal Total Cost: {model.objVal}")
    for t in range(1, T+1):
        print(f"\nTime Period {t}:")
        for g in generators:
            print(f"  Unit {g}: Status={u[g,t].x}, Power={p[g,t].x}, Start-up={v[g,t].x}")
        print(f"  Curtailed Energy: {s[t].x}")
else:
    print("No optimal solution found. Please check the model setup or data.")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 49480 rows, 4200 columns and 145452 nonzeros
Model fingerprint: 0xe83fceb5
Variable types: 1512 continuous, 2688 integer (2688 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [8e+00, 3e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+03]
Presolve removed 32304 rows and 1665 columns
Presolve time: 0.16s
Presolved: 17176 rows, 2535 columns, 52195 nonzeros
Variable types: 1296 continuous, 1239 integer (1239 binary)
Found heuristic solution: objective 1.247278e+07
Found heuristic solution: objective 1.216149e+07
Root relaxation presolved: 12025 rows, 7686 columns, 41893 nonzeros


Root relaxation: objective 9.122799e+06, 1609 iterations, 0.06 seconds (0.12 work units)

    Nodes    |    Current Node    |     Objective Bounds      | 